In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [ ]:
pd.re

In [2]:
url = "https://www.jumia.com.eg/catalog/?q=red+sport+shoes"

In [257]:
content = requests.get(url).content

In [258]:
soup = BeautifulSoup(content)

In [259]:
print(soup.find_all("title"))

[<title>Shop All Products @ #1 Online Store - Enjoy Online Shopping @ Best Prices - Jumia Egypt</title>]


In [15]:
#I'm trying to find where the links of the products are
print(soup.find(text = "https://www.jumia.com.eg/starter-boysgilrs-sneaker-blackred-17727164.html"))

None


In [32]:
#Here i brought the title of a random product in the page to try to find it on the main soup
title_cotent = requests.get("https://www.jumia.com.eg/starter-boysgilrs-sneaker-blackred-17727164.html").content
title_soup = BeautifulSoup(title_cotent)
random_title = title_soup.find("title")

In [43]:
type(random_title.contents)

list

In [44]:
#And it didn't work
soup.find(text = random_title.contents[0])

In [26]:
#I will try to search for the product name
soup.find(text = "Activ Stitched Lace Up Navy Blue & Red Leather Football Shoes").parent.parent
#I found it but i can't find the link of the product in this div

<div class="info"><h3 class="name">Activ Stitched Lace Up Navy Blue &amp; Red Leather Football Shoes</h3><div class="prc">EGP 849.00</div><div class="s-prc-w"><div class="old">EGP 1,199.00</div><div class="bdg _dsct _sm">29%</div></div><svg aria-label="Express Shipping" class="ic xprss" height="10" viewbox="0 0 114 12" width="94"><use xlink:href="https://www.jumia.com.eg/assets_he/images/i-shop-jumia.9f5451c7.svg#express"></use></svg><p class="shipp">Eligible for free delivery</p></div>

from this one i can get (Price, discount, price after discount, product name, shipping type) but i still want 
the product ratings

Now i will try to find all the div with class "info" and hopfilly i will be able to create a list with all producta name in this page

In [260]:
div_list = soup.find_all("div", class_ = "info")

Now i have a list with all divs i can extract all the data i need

In [261]:
div_list[0]

<div class="info"><h3 class="name">Red Knitted Knitted Shawl</h3><div class="prc">EGP 178.00</div><div class="s-prc-w"><div class="old">EGP 300.00</div><div class="bdg _dsct _sm">41%</div></div></div>

In [262]:
#number of products in this page
len(div_list)

40

I will work on one div then i wil make a for loop for the all list

In [221]:
sample = div_list[0]

In [222]:
product_name = sample.find("h3", class_ = "name").contents[0]

In [223]:
# to extract the old price
old_price = sample.find("div", class_ = "old").contents[0]
#I want to remove the EGP and ',' and make it float
old_price = old_price.split()[1]
old_price = float(old_price.replace(",",""))
old_price

260.0

In [224]:
price = sample.find("div", class_ = "prc").contents[0]
#I want to remove the EGP and make it float
price = price.split()[1]
price = float(price.replace(",",""))

In [225]:
discount = sample.find("div", class_ = "bdg _dsct _sm").contents[0]
discount = int(discount.replace("%", ""))

In [226]:
rating = sample.find("div", class_ = "stars _s").contents[0]
rating = float(rating.split()[0])

In [94]:
#I will extract if there is free delivry or not
free_delivery = sample.find("p").contents[0]

Now i will work on converting all of this into a for loop

In [263]:
#Empty lists for the values
names_list = []
old_prices_list = []
new_prices_list = []
discount_list = []
rating_list = []
free_delivery_list = []

In [264]:
for div in div_list:
    product_name = div.find("h3", class_ = "name").contents[0]
    
    price = div.find("div", class_ = "prc").contents[0]
    #I want to remove the EGP and "," to make it float
    price = price.split()[1]
    price = float(price.replace(",",""))
    try:
        old_price = div.find("div", class_ = "old").contents[0]
        old_price = old_price.split()[1]
        old_price = float(old_price.replace(",",""))
    except:
        old_price = price
    
    try:
        discount = div.find("div", class_ = "bdg _dsct _sm").contents[0]
        discount = int(discount.replace("%", ""))
    except:
        discount = 0
    
    try:
        rating = div.find("div", class_ = "stars _s").contents[0]
        rating = float(rating.split()[0])
    except:
        rating = "No rating"
        
    try:
        free_delivery = div.find("p").contents[0]
    except:
        free_delivery = "NA"
    
    #Now i will append these values into new lists
    
    names_list.append(product_name)
    old_prices_list.append(old_price)
    new_prices_list.append(price)
    discount_list.append(discount)
    rating_list.append(rating)
    free_delivery_list.append(free_delivery)

In [265]:
#If this is working the len should give you 40, if not make sure that you run the cell of the empty list first before
# running the cell of the for loop
len(names_list)

40

In [266]:
#Nowi will create the data frame
dict_ = {"product_name": names_list, "old_price": old_prices_list, "discount": discount_list, 
         "current_price": new_prices_list,
         "rating": rating_list, "delivery": free_delivery_list}

In [267]:
df = pd.DataFrame(dict_)

After i created the data set i want to take a look at it to see if any thing needs to be edited

In [268]:
df.head()

,product_name,old_price,discount,current_price,rating,delivery
0,Red Knitted Knitted Shawl,300.0,41,178.0,No rating,NA
1,Ricci Casual Patterned Dark Red Cape For Woman,600.0,50,300.0,5.0,NA
2,Blouse Barn Red Paisley Viscose Kimono Suit,599.0,53,279.0,No rating,NA
3,Velvet Hooded Cloak Long Robe Witch Costume ...,560.0,23,431.0,No rating,NA
4,Velvet Hooded Cloak Long Robe Witch Costume ...,681.0,23,524.0,No rating,NA


In [269]:
df.query("discount == 0")

,product_name,old_price,discount,current_price,rating,delivery
7,Cap Shawl For Women Wool Gogh Red Color High H...,650.0,0,650.0,No rating,NA
10,Lovely Red Cape For Woman,575.0,0,575.0,5.0,NA
16,Casual Cape For Women - Red,950.0,0,950.0,No rating,NA


In [270]:
df["delivery"].value_counts()

NA    40
Name: delivery, dtype: int64

In [271]:
# I wanted to know how many products have no rating
df.query("rating == 'No rating'")["rating"].value_counts()

No rating    35
Name: rating, dtype: int64

In [272]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_name   40 non-null     object 
 1   old_price      40 non-null     float64
 2   discount       40 non-null     int64  
 3   current_price  40 non-null     float64
 4   rating         40 non-null     object 
 5   delivery       40 non-null     object 
dtypes: float64(2), int64(1), object(3)
memory usage: 2.0+ KB


Now i want to create the code that will make the url from the search key word for all of this to work at the first place

In [252]:
#I will convert this into an input while writing the final product on suplime text
key_word = "red capes"

In [253]:
url_key_word = key_word.replace(" ", "+")

In [254]:
url_key_word

'red+capes'

In [255]:
url = "https://www.jumia.com.eg/catalog/?q={}".format(url_key_word)

In [256]:
url

'https://www.jumia.com.eg/catalog/?q=red+capes'